In [8]:
!pip install pandas

In [9]:
import pandas as pd

In [ ]:
train=pd.read_csv('/content/train_dataset.csv')

In [10]:
test=pd.read_csv('/content/test_dataset.csv')

In [ ]:
print("Missing values in X_transformed:")
print(X_transformed.isnull().sum())

print("Missing values in y:")
print(pd.Series(y).isnull().sum())  # Convert y to Series if it's not a DataFrame


In [12]:
X = train.drop(columns=['IncidentGrade'])  # Features
y = train['IncidentGrade']                   # Target variable


In [ ]:
#  drop the missing nan values

train.dropna(inplace=True)
test.dropna(inplace=True)

X = train.drop(columns=['IncidentGrade'])  # Features
y = train['IncidentGrade']                   # Target variable

<ipython-input-23-06daf03686e7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.dropna(inplace=True)


In [25]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

In [24]:
categorical_cols = ['Category', 'EntityType', 'EvidenceRole']  # Categorical columns in your dataset
numerical_cols = X.select_dtypes(include=['int64', 'int32','float64']).columns  # Automatically select numerical columns


In [26]:
# Step 1: Preprocessing - OneHotEncode the categorical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),  # Keep numerical columns as they are
        ('cat', OneHotEncoder(), categorical_cols)  # OneHotEncode categorical columns
    ]
)

# Step 2: Create a pipeline for preprocessing
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)  # Only preprocessing at this stage
])

In [27]:
# Step 3: Transform the features using the pipeline
X_transformed = pipeline.fit_transform(X)

# Step 4: Split data into training and validation sets before SMOTE
X_train, X_val, y_train, y_val = train_test_split(X_transformed, y, test_size=0.2, stratify=y, random_state=42)


In [28]:

# Step 5: Scale the resampled training data
X_train = StandardScaler().fit_transform(X_train)

# Step 6: Scale the validation data using the same scaler fitted on the training set
X_val_scaled = StandardScaler().fit_transform(X_val)


In [29]:
from sklearn.ensemble import RandomForestClassifier

In [30]:
model_pipeline = Pipeline(steps=[
    ('Classifier',RandomForestClassifier(random_state=42))
])

In [31]:
# Step 7: Train the model using the resampled data
model_pipeline.fit(X_train, y_train)


Pipeline(steps=[('Classifier', RandomForestClassifier(random_state=42))])

In [32]:
# Step 8: Evaluate on the validation set
val_predictions = model_pipeline.predict(X_val)

In [36]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score


In [37]:
# Step 3: Make predictions using the trained model
from sklearn.metrics import precision_score, recall_score



# Step 4: Calculate metrics (use y_test for evaluation)
accuracy = accuracy_score(y_val, val_predictions)
f1 = f1_score(y_val, val_predictions, average='macro')  # You can change 'weighted' to 'macro' or 'micro'
recall = recall_score(y_val, val_predictions, average='macro')
precision = precision_score(y_val, val_predictions, average='macro')


In [38]:
# Display the results
print(f"Accuracy: {accuracy:.2f}")
print(f"F1-Score: {f1:.5f}")
print(f"Recall: {recall:.5f}")
print(f"Precision: {precision:.5f}")

Accuracy: 0.81
F1-Score: 0.78397
Recall: 0.76022
Precision: 0.85438


In [40]:
# Step 8: Evaluate on the validation set
val_predictions = model_pipeline.predict(X_val)

# ... other code ...

# Assume you want to create a confusion matrix for the validation set
# Since y_test and test_predictions are not defined, use y_val and val_predictions instead:

#Create confusion matrix using crosstab
confusion = pd.crosstab(y_val, val_predictions, rownames=['Actual'], colnames=['Predicted'], margins=True)

In [42]:
from joblib import dump

dump(model_pipeline, 'random_forest_model.joblib')


['random_forest_model.joblib']

In [ ]:
# the final macro-F1 score, precision, and recall for above model

print(f"Final Macro-F1 Score: {f1:.5f}")
print(f"Final Macro-Precision: {precision:.5f}")
print(f"Final Macro-Recall: {recall:.5f}")

Final Macro-F1 Score: 0.78397
Final Macro-Precision: 0.85438
Final Macro-Recall: 0.76022


In [ ]:
#feacher importance

import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from joblib import dump
import numpy as np

!pip install pandas imblearn scikit-learn joblib

try:
    train = pd.read_csv('/content/train_dataset.csv')
    test = pd.read_csv('/content/test_dataset.csv')
except FileNotFoundError:
    print("Error: One or both of the CSV files ('train_dataset.csv', 'test_dataset.csv') were not found.")
    exit()


X = train.drop(columns=['IncidentGrade'])  # Features
y = train['IncidentGrade']                   # Target variable

#Handle missing values before splitting
train.dropna(inplace=True)
X = train.drop(columns=['IncidentGrade'])
y = train['IncidentGrade']

categorical_cols = ['Category', 'EntityType', 'EvidenceRole']
numerical_cols = X.select_dtypes(include=['int64', 'int32', 'float64']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

pipeline = Pipeline(steps=[('preprocessor', preprocessor)])
X_transformed = pipeline.fit_transform(X)


X_train, X_val, y_train, y_val = train_test_split(X_transformed, y, test_size=0.2, stratify=y, random_state=42)

#Scaling after splitting and before model training
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)


model_pipeline = Pipeline(steps=[('Classifier',RandomForestClassifier(random_state=42))])
model_pipeline.fit(X_train, y_train)

val_predictions = model_pipeline.predict(X_val)

accuracy = accuracy_score(y_val, val_predictions)
f1 = f1_score(y_val, val_predictions, average='macro')
recall = recall_score(y_val, val_predictions, average='macro')
precision = precision_score(y_val, val_predictions, average='macro')

print(f"Accuracy: {accuracy:.2f}")
print(f"F1-Score: {f1:.5f}")
print(f"Recall: {recall:.5f}")
print(f"Precision: {precision:.5f}")

confusion = pd.crosstab(y_val, val_predictions, rownames=['Actual'], colnames=['Predicted'], margins=True)
print(confusion)

dump(model_pipeline, 'random_forest_model.joblib')

print(f"Final Macro-F1 Score: {f1:.5f}")
print(f"Final Macro-Precision: {precision:.5f}")
print(f"Final Macro-Recall: {recall:.5f}")

importances = model_pipeline.named_steps['Classifier'].feature_importances_
feature_names = list(pipeline.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(categorical_cols))
numerical_feature_names = list(numerical_cols)
feature_names = numerical_feature_names + feature_names
feature_importances = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)
feature_importances

Accuracy: 0.91
F1-Score: 0.90204
Recall: 0.89250
Precision: 0.91560
Predicted       BenignPositive  FalsePositive  TruePositive    All
Actual                                                            
BenignPositive           20339            354           513  21206
FalsePositive             1373           8746           380  10499
TruePositive              1696            269         15173  17138
All                      23408           9369         16066  48843
Final Macro-F1 Score: 0.90204
Final Macro-Precision: 0.91560
Final Macro-Recall: 0.89250


,Feature,Importance
1,OrgId,0.187553
2,IncidentId,0.134831
3,AlertId,0.096786
4,DetectorId,0.094540
5,AlertTitle,0.074191
...,...,...
54,EntityType_Blob,0.000004
68,EntityType_MailboxConfiguration,0.000004
52,EntityType_ActiveDirectoryDomain,0.000001
63,EntityType_KubernetesCluster,0.000000


In [ ]:
#   Compare the performance on the test set to the baseline model and initial validation results to ensure consistency and improvement.

# Preprocess the test data using the same pipeline fitted on the training data
X_test = test.drop(columns=['IncidentGrade'])
X_test_transformed = pipeline.transform(X_test)

# Scale the test data using the same scaler fitted on the training data
X_test_scaled = scaler.transform(X_test_transformed)

# Make predictions on the test set
test_predictions = model_pipeline.predict(X_test_scaled)

# Evaluate the model on the test set
test_accuracy = accuracy_score(test['IncidentGrade'], test_predictions)
test_f1 = f1_score(test['IncidentGrade'], test_predictions, average='macro')
test_recall = recall_score(test['IncidentGrade'], test_predictions, average='macro')
test_precision = precision_score(test['IncidentGrade'], test_predictions, average='macro')

print("\nTest Set Evaluation:")
print(f"Accuracy: {test_accuracy:.2f}")
print(f"F1-Score: {test_f1:.5f}")
print(f"Recall: {test_recall:.5f}")
print(f"Precision: {test_precision:.5f}")

# Compare test set performance to validation set performance and baseline
print("\nPerformance Comparison:")
print(f"Validation F1-Score: {f1:.5f}")
print(f"Test F1-Score: {test_f1:.5f}")

#  You would replace 'baseline_f1' with the actual baseline F1 score.
baseline_f1 = 0.5  # Example baseline
print(f"Baseline F1-Score: {baseline_f1:.5f}")


# Check for consistency and improvement
if test_f1 > baseline_f1 and test_f1 >= f1 :
    print("Model performance on the test set is consistent with validation results and improves upon the baseline.")
elif test_f1 > baseline_f1 and test_f1 < f1:
    print("Model performance on the test set improves upon the baseline, but is lower than validation results. Possible overfitting.")
else:
    print("Model performance on the test set is not better than the baseline. Further investigation is needed.")

#Further analysis (optional)
#Create a confusion matrix for the test set
test_confusion = pd.crosstab(test['IncidentGrade'], test_predictions, rownames=['Actual'], colnames=['Predicted'], margins=True)
print("\nTest Set Confusion Matrix:")
test_confusion


Test Set Evaluation:
Accuracy: 0.87
F1-Score: 0.86507
Recall: 0.85671
Precision: 0.87749

Performance Comparison:
Validation F1-Score: 0.90204
Test F1-Score: 0.86507
Baseline F1-Score: 0.50000
Model performance on the test set improves upon the baseline, but is lower than validation results. Possible overfitting.

Test Set Confusion Matrix:


Predicted,BenignPositive,FalsePositive,TruePositive,All
Actual,,,,
BenignPositive,82506,3311,3152,88969
FalsePositive,7642,35532,2476,45650
TruePositive,8526,1743,65478,75747
All,98674,40586,71106,210366
